In [1]:
trainFolders = [
     'ARMS',
     'AisazuNihaIrarenai',
     'AkkeraKanjinchou',
     'Akuhamu',
     'AosugiruHaru',
     'AppareKappore',
     'Arisa',
     'BEMADER_P',
     'BakuretsuKungFuGirl',
     'Belmondo',
     'BokuHaSitatakaKun',
     'BurariTessenTorimonocho',
     'ByebyeC-BOY',
     'Count3DeKimeteAgeru',
     'DollGun',
     'Donburakokko',
     'DualJustice',
     'EienNoWith',
     'EvaLady',
     'EverydayOsakanaChan',
     'GOOD_KISS_Ver2',
     'GakuenNoise',
     'GarakutayaManta',
     'GinNoChimera',
     'Hamlet',
     'HanzaiKousyouninMinegishiEitarou',
     'HaruichibanNoFukukoro',
     'HarukaRefrain',
     'HealingPlanet',
     "UchiNoNyan'sDiary",
     'UchuKigekiM774',
     'UltraEleven',
     'UnbalanceTokyo',
     'WarewareHaOniDearu',
     'YamatoNoHane',
     'YasasiiAkuma',
     'YouchienBoueigumi',
     'YoumaKourin',
     'YukiNoFuruMachi',
     'YumeNoKayoiji',
     'YumeiroCooking',
     'TotteokiNoABC',
     'ToutaMairimasu',
     'TouyouKidan',
     'TsubasaNoKioku'
]


In [2]:
import numpy as np   
import utils as U   
import glob
from PIL import Image
from pathlib import Path   
from sklearn.model_selection import KFold

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import models as M
import numpy as np
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import callbacks
import keras
import pickle

In [3]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, fold, patches = 40, train = False, dims=(128,128), shuffle=True):
        self.shuffle = shuffle
        self.patches = patches
        self.train = train
        self.dims = dims
        self.fold = list(KFold(n_splits = 5, shuffle = True, random_state=42).split(trainFolders, trainFolders))[fold]
        self.init_images()
        self.on_epoch_end()

    def init_images(self):
        self.images = []
        for f in glob.glob("/data/anime/masks/**/*.cache"):
            index = trainFolders.index(Path(f).parent.name)
            if index in self.fold[0] and self.train:
                self.images.append(f)
            elif index in self.fold[1] and not self.train:
                self.images.append(f)
        #self.images = self.images[0:3]
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images)))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch

        # Generate data
        X, y = self.__data_generation([self.images[index]])

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, files):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        images = []
        masks = []
        
        for f in files:
            mask = np.array(Image.open(f))
            mask[mask >= 3] -= 3
            mask = (mask != 0) * 255
            img = np.asarray(Image.open('/data/anime/manga/Manga109_2017_09_28/images/' + Path(f).parent.name + '/' + Path(f).stem + '.jpg').convert('L'))
            images.append(img)
            masks.append(mask)
        
        patches_image, patches_masks = U.extract_random(images, masks, *self.dims, self.patches)
        
        patches_image /= 255.
        patches_masks /= 255.       
        
        patches_image  = np.expand_dims(patches_image,  axis = 3)
        patches_masks = np.expand_dims(patches_masks, axis = 3)
        return patches_image, patches_masks
        

In [ ]:
def train(index):
    train = DataGenerator(index, train=True)
    test = DataGenerator(index, train=False)
    print('Dataset Prepared')

    model = M.BCDU_net_D3(input_size = (128, 128, 1))
    model.summary()

    print('Training')

    nb_epoch = 20

    mcp_save = ModelCheckpoint('weight_text'+str(index)+'.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

    history = model.fit(x=train,
                  epochs=nb_epoch,
                  verbose=1,
                  validation_data=test, callbacks=[mcp_save, reduce_lr_loss] )

    print('Trained model saved')
    with open('fold' + str(index), 'wb') as file_pi:
            pickle.dump(history.history, file_pi)

In [ ]:
train(0)

Dataset Prepared


W0904 03:00:48.262387 139725218301696 callbacks.py:2323] `epsilon` argument is deprecated and will be removed, use `min_delta` instead.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 64)   0           conv2d_1[0][0]                   
_______________________________________________________________________________________

Epoch 1/20
360/360 [==============================] - 292s 811ms/step - loss: 0.1528 - accuracy: 0.9642 - val_loss: 0.1258 - val_accuracy: 0.9681
Epoch 2/20
360/360 [==============================] - 289s 803ms/step - loss: 0.1127 - accuracy: 0.9664 - val_loss: 0.1294 - val_accuracy: 0.9683
Epoch 3/20
360/360 [==============================] - 290s 807ms/step - loss: 0.1005 - accuracy: 0.9699 - val_loss: 0.1030 - val_accuracy: 0.9703
Epoch 4/20
360/360 [==============================] - 289s 802ms/step - loss: 0.0884 - accuracy: 0.9733 - val_loss: 0.1331 - val_accuracy: 0.9682
Epoch 5/20
278/360 [======================>.......] - ETA: 1:00 - loss: 0.0852 - accuracy: 0.9747

In [ ]:
train(1)

In [ ]:
train(2)

In [ ]:
train(3)

In [ ]:
train(4)

In [ ]:
for fold in range(0, 5):
    model = M.BCDU_net_D3(input_size = (128, 128,1))
    model.load_weights('weight_text'+str(fold)+'.hdf5')
    test = DataGenerator(fold, train=False)

    for f in test.images:
        img = np.asarray(Image.open('/data/anime/manga/Manga109_2017_09_28/images/' + Path(f).parent.name + '/' + Path(f).stem + '.jpg').convert('L'))
        patches , new_h, new_w = U.extract_ordered_overlap(img, 128, 128, 64, 64)
        patches     = np.expand_dims(patches,  axis = 3)
        predictions = model.predict(patches, batch_size= 40, verbose=1)
        estimated   = U.recompone_overlap(predictions[:,:,:,0], new_h, new_w, 64, 64)
        estimated   = np.where(estimated >= 0.7, 1, 0)
        save_path = '/data/anime/predictions/' + Path(f).parent.name
        Path(save_path).mkdir(parents=True, exist_ok=True)
        Image.fromarray(estimated.astype(np.uint8)).save(save_path  + '/' + Path(f).stem + '.png')

In [104]:
for fold in range(0, 5):
    test = DataGenerator(fold, train=False)
    count = 0
    for f in test.images:
        pred = open_mask('/data/anime/predictions/' + Path(f).parent.name + '/' + Path(f).stem + '.png').px
        count += pred.max()
    print(fold, count)

0 tensor(90.)
1 tensor(0.)
2 tensor(0.)
3 tensor(90.)
4 tensor(90.)
